In [1]:
# =========================================================
# 1️⃣ KÜTÜPHANELER
# =========================================================
import pandas as pd
import numpy as np

from scipy.sparse import csr_matrix
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

import pickle
from tqdm import tqdm


# =========================================================
# 2️⃣ VERİYİ YÜKLE
# =========================================================
df = pd.read_csv("ratings_Electronics (1).csv", header=None)
df.columns = ['userId', 'productId', 'rating', 'timestamp']

df = df[['userId', 'productId', 'rating']]
print("Orijinal veri boyutu:", df.shape)


# =========================================================
# 3️⃣ RAM DOSTU FİLTRELEME (ZORUNLU)
# =========================================================

# Aktif kullanıcılar (en az 10 rating)
active_users = df['userId'].value_counts()
active_users = active_users[active_users >= 10].index
df = df[df['userId'].isin(active_users)]

# Popüler ürünler (en az 10 rating)
popular_products = df['productId'].value_counts()
popular_products = popular_products[popular_products >= 10].index
df = df[df['productId'].isin(popular_products)]

print("Filtrelenmiş veri boyutu:", df.shape)


# =========================================================
# 4️⃣ TRAIN / TEST AYRIMI
# =========================================================
train_df, test_df = train_test_split(
    df,
    test_size=0.2,
    random_state=42
)


# =========================================================
# 5️⃣ ITEM–USER SPARSE MATRIX
# =========================================================

# Kategorik kodlama
user_codes = train_df['userId'].astype("category").cat.codes
product_codes = train_df['productId'].astype("category").cat.codes

user_categories = train_df['userId'].astype("category").cat.categories
product_categories = train_df['productId'].astype("category").cat.categories

item_user_matrix = csr_matrix(
    (train_df['rating'], (product_codes, user_codes))
)

print("Sparse matrix shape:", item_user_matrix.shape)


# =========================================================
# 6️⃣ ITEM–ITEM COSINE SIMILARITY (ÇÖKMEZ)
# =========================================================
item_similarity = cosine_similarity(
    item_user_matrix,
    dense_output=False
)

print("Item similarity matrix hazır")


# =========================================================
# 7️⃣ ITEM-BASED TAHMİN FONKSİYONU
# =========================================================
def predict_rating_item_based(user_id, product_id):
    if user_id not in user_categories or product_id not in product_categories:
        return np.nan

    user_idx = user_categories.get_loc(user_id)
    product_idx = product_categories.get_loc(product_id)

    similarities = item_similarity[product_idx].toarray().flatten()

    user_ratings = item_user_matrix[:, user_idx].toarray().flatten()

    mask = user_ratings > 0
    if mask.sum() == 0:
        return np.nan

    return np.dot(similarities[mask], user_ratings[mask]) / np.sum(similarities[mask])


# =========================================================
# 8️⃣ MODEL SKORU (RMSE) — MODELİ KAYDETMEDEN ÖNCE
# =========================================================
y_true = []
y_pred = []

print("RMSE hesaplanıyor...")

for _, row in tqdm(test_df.iterrows(), total=len(test_df)):
    pred = predict_rating_item_based(
        row['userId'],
        row['productId']
    )
    if not np.isnan(pred):
        y_true.append(row['rating'])
        y_pred.append(pred)

rmse = np.sqrt(mean_squared_error(y_true, y_pred))
print("\n📊 MODEL RMSE:", rmse)


# =========================================================
# 9️⃣ MODELİ KAYDET (EN SON!)
# =========================================================
with open("amazon_item_based_model.pkl", "wb") as f:
    pickle.dump(
        {
            "item_user_matrix": item_user_matrix,
            "item_similarity": item_similarity,
            "user_categories": user_categories,
            "product_categories": product_categories
        },
        f
    )

print("✅ Model başarıyla kaydedildi.")

Orijinal veri boyutu: (7824482, 3)
Filtrelenmiş veri boyutu: (722237, 3)
Sparse matrix shape: (20972, 62991)
Item similarity matrix hazır
RMSE hesaplanıyor...


C:\Users\wfi-7\AppData\Local\Temp\ipykernel_11844\2304117532.py:100: RuntimeWarning: invalid value encountered in scalar divide
  return np.dot(similarities[mask], user_ratings[mask]) / np.sum(similarities[mask])
100%|█████████████████████████████████████████████████████████████████████████| 144448/144448 [03:24<00:00, 705.61it/s]



📊 MODEL RMSE: 1.2271461280859346
✅ Model başarıyla kaydedildi.
